# Lab 2 Sieci Neuronowe w Automatyce -  algorytm wstecznej propagacji błędów z biblioteką NumPy

Imię i nazwisko: ...

Wzory na podstawie: https://www.deeplearning.ai/deep-learning-specialization/ Neural Networks and Deep Learning

#### Punktacja (do 10 pkt.)
* Poprawnie wypełniony notebook 6 pkt.
* Odpowiedzi na pytania na końcu 3 pkt. (1 pytanie = 1 punkt)
* Bonusy dla chętnych do 2 pkt.
* Suma z laboratorium nie może wynosić więcej niż 10 pkt. (wyniki większe niż 10 są obcinane do 10)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt # biblioteka do wykresów
from mpl_toolkits.mplot3d import Axes3D # do wykresu 3D

from sklearn.datasets import make_classification # generacja danych, sklearn - bardzo przydatna biblioteka do uczenia maszynowego

## Generacja danych

Problem klasyfikacji binarnej (kropka żółta czy fioletowa?) dla trzech zmiennych wejściowych $x_1$, $x_2$, $x_3$

In [ ]:
m = 300
data = make_classification(n_samples=m, n_features=3, n_informative=3, n_redundant=0, n_classes=2, random_state=0)

In [ ]:
X = data[0]
Y = data[1]

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
cax = ax.scatter(X[:,0], X[:,1], X[:,2], c=Y.transpose())
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')
ax.set_zlabel('$x_3$')
fig.colorbar(cax)
plt.show()

# Sieć neuronowa

In [1]:
%%html
<iframe src="https://drive.google.com/file/d/1s3mGcho6uv2VdMb4CBsw686qwhpp2mRw/preview" width="640" height="480"></iframe>

$$ 
\begin{bmatrix}
w_{11}^{[1]} & w_{12}^{[1]}  & w_{13}^{[1]}\\
\vdots & \vdots & \vdots \\
\vdots & \vdots & \vdots \\
w_{41}^{[1]} & w_{42}^{[1]}  & w_{43}^{[1]}
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3
\end{bmatrix}
+
\begin{bmatrix}
b_1^{[1]} \\
b_2^{[1]} \\
b_3^{[1]} \\
b_4^{[1]}
\end{bmatrix}
=
\begin{bmatrix}
z_1^{[1]} \\
z_2^{[1]} \\
z_3^{[1]} \\
z_4^{[1]}
\end{bmatrix}
$$

$$ z^{[1]} = W^{[1]}x + b^{[1]} $$

$$ a^{[1]} = \varphi^{[1]}(z^{[1]}) $$

$$ z^{[2]}  = W^{[2]}a^{[1]} + b^{[2]} $$

$$ a^{[2]} = \varphi^{[2]}(z^{[2]}) $$

#### Incjalizacja wartości wag

In [ ]:
np.random.seed(0)

In [ ]:
def initialize():
    # wartości W losujemy z rozkładu normalnego o średniej 0 i odchyleniu standardowym 0.1, wartości b ustawiamy na 0
    W1 = np.random.normal(0, 0.1, size=(4, 3))
    b1 = np.zeros(shape=(4, 1))
    W2 = # TODO
    b2 = # TODO
    return W1, b1, W2, b2

In [ ]:
W1, b1, W2, b2 = initialize()

In [ ]:
print('W1 = ', W1)
print('b1 = ', b1)
print('W2 = ', W2)
print('b2 = ', b2)

In [ ]:
assert(W1.shape == (4, 3))
assert(b1.shape == (4, 1))
assert(W2.shape == (1, 4))

In [ ]:
np.testing.assert_almost_equal(W2, [[0.07610377, 0.0121675, 0.04438632, 0.03336743]])

### Funkcje aktywacji

In [ ]:
def sigmoid(Z):
    # sigmoidalna funkcja aktywacji dla macierzy Z
    return # TODO

def leaky_relu(Z):
    # funkcja aktywacji leaky relu dla macierzy Z (leaky_relu(Z) = max(0.01 * Z, Z))
    return # TODO

def leaky_relu_grad(Z):
    # pochodna funkcji leaky relu dla macierzy Z
    # https://numpy.org/doc/stable/reference/generated/numpy.where.html
    return # TODO

In [ ]:
testZ = np.array([[-3, -2, -1], [0, 1, 2]])
print('Sigmoid:')
print(sigmoid(testZ))
print('Leaky ReLu:')
print(leaky_relu(testZ))
print('Leaky ReLu pochodna:')
print(leaky_relu_grad(testZ))

In [ ]:
np.testing.assert_almost_equal(sigmoid(testZ), [[0.04742587, 0.11920292, 0.26894142], [0.5, 0.73105858, 0.88079708]])
np.testing.assert_almost_equal(leaky_relu(testZ), [[-0.03, -0.02, -0.01], [0, 1, 2]])
np.testing.assert_almost_equal(leaky_relu_grad(testZ), [[0.01, 0.01, 0.01], [1., 1., 1.]])

## Propagacja w przód

Wektoryzacja dla wielu przykładów

$$ X = \begin{bmatrix}
x_1 ^{(1)} & x_1 ^{(2)} & \dots & x_1 ^{(m)}\\
x_2 ^{(1)} & x_2 ^{(2)} & \dots & x_2 ^{(m)}\\
x_3 ^{(1)} & x_3 ^{(2)} & \dots & x_3 ^{(m)}
\end{bmatrix} $$

$$ Z^{[1]} =  W^{[1]}X + b^{[1]}  $$
$$ A^{[1]} = \varphi^{[1]}(Z^{[1]}) $$
$$ Z^{[2]} = W^{[2]}A^{[1]} + b^{[2]}  $$
$$ A^{[2]} = \varphi^{[2]}(Z^{[2]}) $$
$$ \hat{Y} = A^{[2]} $$

In [ ]:
X.shape

In [ ]:
Xt = X.transpose()

In [ ]:
Xt.shape

In [ ]:
Y = Y.reshape(1, m)

In [ ]:
def forward(X, W1, W2, b1, b2):
    # wyznaczamy wyjście sieci i wartości w warstwach ukrytych
    Z1 = np.dot(W1, X) + b1
    A1 = # TODO
    Z2 = # TODO
    A2 = # TODO
    Y_hat = # TODO
    return Z1, A1, Z2, A2, Y_hat

In [ ]:
Z1, A1, Z2, A2, Y_hat = forward(Xt, W1, W2, b1, b2)

In [ ]:
assert(Z1.shape == (4, m))
assert(Z2.shape == (1, m))
assert(Y_hat.shape == (1, m))

In [ ]:
np.testing.assert_almost_equal(Y_hat[0][:3], [0.50025724, 0.49987587, 0.50083385])

### Funkcja kosztu

Dla klasyfikacji binarnej stosujemy entropię krzyżową
$$ -\frac{1}{m} \sum_{i=1}^m \left( y_i \log(\hat{y}_i) + (1 - y_i)\log(1-\hat{y}_i)\right) $$

In [ ]:
def J(Y, Y_hat):
    # binarna entropia krzyżowa
    # proszę zastosować logarytm naturalny (np.log)
    return # TODO

In [ ]:
assert(round(J(np.array([[1, 0, 1]]), np.array([[0.98, 0.1, 0.5]])), 4) == 0.2729)

## Wyznaczanie pochodnych metodą wstecznej propagacji błędów

$$ \frac{\partial J}{\partial Z^{[2]}} = A^{[2]} - Y$$
$$ \frac{\partial J}{\partial W^{[2]}} =\frac{1}{m} \frac{dJ}{\partial Z^{[2]}} A^{[1]T} $$
$$ \frac{\partial J}{\partial b^{[2]}} = \frac{1}{m} np.sum(\frac{dJ}{\partial Z^{[2]}}, axis = 1, keepdims=True) $$
$$\frac{dJ}{\partial Z^{[1]}} = W^{[2]T}\frac{dJ}{\partial Z^{[2]}} * \varphi^{[1]{\prime}}(z_1)$$

$$ \frac{\partial J}{\partial W^{[1]}} = \frac{dJ}{\partial Z^{[1]}} X^T $$
$$ \frac{\partial J}{\partial b^{[1]}} = \frac{1}{m} np.sum(\frac{dJ}{\partial Z^{[1]}}, axis = 1, keepdims=True) $$

In [ ]:
def calculate_gradients(Y, A2, A1, Z1, W2, W1, b2, b1, X, m):
    dZ2 = A2 - Y
    dW2 = # TODO
    db2 = # TODO
    dZ1 = # TODO
    dW1 = # TODO
    db1 = # TODO
    return dW2, db2, dW1, db1

In [ ]:
dW2, db2, dW1, db1 = calculate_gradients(Y, A2, A1, Z1, W2, W1, b2, b1, Xt, m)

In [ ]:
np.testing.assert_almost_equal(dW2, [[-0.02540348, -0.03184805, -0.00094008, -0.01338454]])
np.testing.assert_almost_equal(db2, [[0.00116861]])
np.testing.assert_almost_equal(dW1, [[-2.12742532, -1.1236145 , -1.63209061], [-0.26206997, -0.40640333, -0.18799057],
                                     [-0.75340629, -2.36955757, -2.24738247],[-1.57469511,  0.94111372, -0.56991805]])
np.testing.assert_almost_equal(db1, [[-0.00156242], [0.00022612], [-0.00021417], [-0.00262392]])

## Algorytm największego spadku gradientu

1. Przypisz wagom losowe wartości początkowe
2. Wyznacz wyjście sieci $\hat{Y}$
3. Wyznacz wartość funkcji kosztu $J$ 
4. $$ dW = \frac{\partial J}{\partial W} \qquad db = \frac{\partial J}{\partial w}$$
$$ W = W - \alpha dW $$
$$ b = b - \alpha db $$
5. dopóki nie warunek stopu idź do 2

In [ ]:
def update(W2, W1, b2, b1, dW2, dW1, db2, db1, alpha=0.001):
    W2 = W2 - alpha * dW2
    W1 = # TODO
    b2 = # TODO
    b1 = # TODO
    return W2, W1, b2, b1

In [ ]:
W2, W1, b2, b1 = update(W2, W1, b2, b1, dW2, dW1, db2, db1)

In [ ]:
assert(W2.shape == (1, 4))

### Dokładność

Wyjściem sieci jest liczba z zakresu 0-1. Dla $\hat{y}^{(i)} > 0.5$ przewidujemy 1 dla pozostałych 0.
Dokładność (accuracy) jest procentem przykładów, które zostały przewidziane przez sieć prawidłowo (dla których $\hat{y}^{(i)} = y^{(i)}$)

In [ ]:
def accuracy(Y, Y_hat):
    return # TODO

In [ ]:
assert(accuracy(np.array([[1, 0, 1, 0]]), np.array([[0.98, 0.1, 0.51, 0.7]])) == 75)

## I w końcu - uczenie sieci

In [ ]:
# inicjalizacja wag
W1, b1, W2, b2 = initialize()
# lista do zapisu wartości funkcji kosztu w kolejnych krokach uczenia (na początku pusta)
J_history = []
# lista do zapisu dokładności w kolejnych krokach (na początku pusta)
acc_history =[]
for i in range(100000):
    Z1, A1, Z2, A2, Y_hat = forward(Xt, W1, W2, b1, b2)
    J_history.append(J(Y, Y_hat))
    acc_history.append(accuracy(Y, Y_hat))
    dW2, db2, dW1, db1 = calculate_gradients(Y, A2, A1, Z1, W2, W1, b2, b1, Xt, m)
    W2, W1, b2, b1 = update(W2, W1, b2, b1, dW2, dW1, db2, db1, alpha=0.001)

In [ ]:
plt.plot(J_history)
plt.title('Funkcja kosztu w zależności od iteracji')
plt.show()

In [ ]:
plt.plot(acc_history)
plt.title('Dokładność w zależności od iteracji')
plt.show()

In [ ]:
# dokładność na koniec
acc_history[-1]

### Predykcja dla nowych danych

In [ ]:
# punkt do sprawdzenia
x_test1 = np.array([[3, 3, 0]])

In [ ]:
# nowy punkt zaznaczony czerwoną kropką
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
cax = ax.scatter(X[:,0], X[:,1], X[:,2], c=Y.reshape(m,))
ax.scatter(x_test1[0][0], x_test1[0][1], x_test1[0][2], c='r')
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')
ax.set_zlabel('$x_3$')
fig.colorbar(cax)
plt.show()

In [ ]:
# predykacja bliska 1 oznacza żółtą kropkę, bliska 0 fioletową kropkę
Z1, A1, Z2, A2, Y_hat = forward(x_test1.transpose(), W1, W2, b1, b2)
print(Y_hat)

## Pytania
1. Jak wpływa wartość $\alpha$ na przebieg uczenia? (proszę spróbować różnych wartości)
2. Czy przy takiej samej wartości $\alpha$ zawsze dostajemy taki sam wynik? Dlaczego?
3. Czy uzyskane wyniki pozwalają stwierdzić, że sieć działa dobrze?

Tu proszę wpisać swoje odpowiedzi:
1. ...
2. ...
3. ...

## Dla chętnych
(proszę sobie zapisać notebook pod nową nazwą do eksperymentów i pozmieniać wybrane)
1. (łatwe) Inna funkcja aktywcji warstwy ukrytej
2. (łatwe) Zmiana liczby neuronów w warstwie ukrytej
3. (średnie) Dodanie kolejnej warstwy ukrytej
4. (trudne - trzeba przeliczyć wzory) Klasyfikacja dla Y o trzech możliwych wartościach